# uSID allocation computation tool
This notebook proposes a uSID allocation computation tool.

- The uSID Block limitations are different for DNX1 platforms. Specify here if a DNX1 platform is used.

In [15]:
import ipaddress

is_dnx1 = input('DNX1 platform? (y/n - default: y): ' or 'y')
if is_dnx1 in 'yY':
    is_dnx1 = True
else:
    is_dnx1 = False
print(f'DNX1 platform?: {"yes" if is_dnx1 else "no"}')

DNX1 platform? (y/n - default: y): 
DNX1 platform?: yes


----
- Specify the SID format. Currently only uSID format F3216 is supported.

In [16]:
sid_format = input('SID format (default: f3216): ').lower() or 'f3216'
if sid_format == 'f3216':
    block_size = 32
    usid_size = 16
else:
    raise Exception (f'Unknown SID format "{sid_format}"')
print(f'SID format: {sid_format}')

SID format (default: f3216): 
SID format: f3216


----
- Specify the SRv6 Space as an IPv6 prefix. Prefix length must be within range \[ block_size-8, block_size-1 \].

In [17]:
srv6_space = ipaddress.IPv6Network(input('SRv6 Space (default: fcbb:bb00::/24): ') or 'fcbb:bb00::/24')
assert srv6_space.prefixlen >= block_size - 8 and srv6_space.prefixlen <= block_size, \
    f'Invalid SRv6 Space prefix length (<{block_size - 8} or >{block_size})'
# check if bits 24 and 25 (for F3216) are 0 for DNX1
if is_dnx1:
    assert (int(srv6_space.network_address) >> (128 - block_size + 6) & 3) == 0, \
        f'DNX1 requires bits {block_size - 8}, {block_size - 7} for this format being zero'
print(f'SRv6 Space: {srv6_space}')

SRv6 Space (default: fcbb:bb00::/24): 
SRv6 Space: fcbb:bb00::/24


----
- Show the number of Blocks in this SRv6 Space, taking the platform limitation into account.

In [18]:
bits_for_blocks = block_size - srv6_space.prefixlen
nr_of_blocks = 2**bits_for_blocks
if is_dnx1:
    # DNX1 supports 64 Blocks, but the last Block is used for SRTE recycling => 63 Blocks left
    bits_for_blocks = min(6, bits_for_blocks)
    nr_of_blocks = min(63, nr_of_blocks)
print(f'Number of Blocks: {nr_of_blocks}')

Number of Blocks: 63


----
- Specify how many algorithms will be used. The number of algorithms must be a multiple of two (2, 4, 8, 16) since this will result in a common prefix grouping Blocks, one of each algo.
    - The algo will be encoded in the LSBs of the Block prefix.
- Each algo must get the same number of Blocks, which may result in leftover Blocks if the number of Blocks is not divisble by the number of algos.

In [19]:
nr_of_algos = int(input('Number of algos (default: 2): ') or 2)
assert nr_of_algos > 0, "Number of algos must be at least 1"
assert nr_of_algos & (nr_of_algos - 1) == 0, "Number of algos must be a power of 2"
assert nr_of_algos <= nr_of_blocks, "Not enough Blocks for this number of algos"
print(f'Number of algos: {nr_of_algos}')
bits_for_algos = (nr_of_algos).bit_length()-1
nr_of_blocks_per_algo = nr_of_blocks // nr_of_algos
print(f'Number of Blocks per algo: {nr_of_blocks_per_algo}')
if nr_of_blocks % nr_of_algos:
    print(f'Number of leftover/unusable Blocks: {nr_of_blocks % nr_of_algos}')

Number of algos (default: 2): 
Number of algos: 2
Number of Blocks per algo: 31
Number of leftover/unusable Blocks: 1


----
- Specify where the algorithm bits are in the LSBs (or MSBs) of the Block ID. Default: LSB.

In [20]:
# TODO: generalize by using algo bit position
algo_in_lsb = input('algo bits in LSBs? (y/n - default: y): ' or 'y')
if algo_in_lsb in 'yY':
    algo_in_lsb = True
else:
    algo_in_lsb = False
print(f'algo bits in LSBs?: {"yes" if algo_in_lsb else "no"}')

algo bits in LSBs? (y/n - default: y): 
algo bits in LSBs?: yes


----
- List the Block prefixes in this SRv6 Space per algo.

In [21]:
for algo_to_print in range(nr_of_algos):
    if algo_to_print is None:
        print(f'All available Blocks')
        print(f'--------------------')
    else:
        print(f'Blocks for Algo {algo_to_print}')
        print(f'------------------')        
    algo_count = [0]*nr_of_algos
    for idx, block in enumerate(srv6_space.subnets(new_prefix=block_size)):
        # which algo?
        # TODO: generalize by using algo bit position
        if algo_in_lsb:
            algo = idx & (nr_of_algos - 1)
        else:
            algo = (idx >> (bits_for_blocks - bits_for_algos)) & (nr_of_algos - 1)
        algo_count[algo] += 1
        # reach max number of Blocks per algo?
        if algo_count[algo] > (nr_of_blocks // nr_of_algos):
            algo = "leftover"
        if algo_to_print is None:
            print(f'Block ID [{idx}] Algo ID [{algo}]: {block}')
        elif algo_to_print == algo:
            print(f'Block ID [{idx}]: {block}')
        if idx >= nr_of_blocks-1:
            break
    print()

Blocks for Algo 0
------------------
Block ID [0]: fcbb:bb00::/32
Block ID [2]: fcbb:bb02::/32
Block ID [4]: fcbb:bb04::/32
Block ID [6]: fcbb:bb06::/32
Block ID [8]: fcbb:bb08::/32
Block ID [10]: fcbb:bb0a::/32
Block ID [12]: fcbb:bb0c::/32
Block ID [14]: fcbb:bb0e::/32
Block ID [16]: fcbb:bb10::/32
Block ID [18]: fcbb:bb12::/32
Block ID [20]: fcbb:bb14::/32
Block ID [22]: fcbb:bb16::/32
Block ID [24]: fcbb:bb18::/32
Block ID [26]: fcbb:bb1a::/32
Block ID [28]: fcbb:bb1c::/32
Block ID [30]: fcbb:bb1e::/32
Block ID [32]: fcbb:bb20::/32
Block ID [34]: fcbb:bb22::/32
Block ID [36]: fcbb:bb24::/32
Block ID [38]: fcbb:bb26::/32
Block ID [40]: fcbb:bb28::/32
Block ID [42]: fcbb:bb2a::/32
Block ID [44]: fcbb:bb2c::/32
Block ID [46]: fcbb:bb2e::/32
Block ID [48]: fcbb:bb30::/32
Block ID [50]: fcbb:bb32::/32
Block ID [52]: fcbb:bb34::/32
Block ID [54]: fcbb:bb36::/32
Block ID [56]: fcbb:bb38::/32
Block ID [58]: fcbb:bb3a::/32
Block ID [60]: fcbb:bb3c::/32

Blocks for Algo 1
------------------


----
- Specify the Set prefix length. This length is a trade-off between allocation granularity and summarization gain. Typically /40 Sets are used. A general recommendation for IPv6 prefix allocation is to align to nibble boundaries (i.e., for Sets: /36, /40, /44) for reasons of human legibility of the prefix.

In [22]:
set_size = int(input('Set prefix length (default: 40): ') or 40)
assert set_size >= block_size + 3, "Set Size too small"
sets_per_block = 2 ** (set_size - block_size)
print(f'Set prefix length: {set_size}')

Set prefix length (default: 40): 
Set prefix length: 40


----
- Compute the Set prefixes. Display the number of available Sets in GIB and the number of uSIDs per Set. Use the IOS XR default GIB/LIB sub-division (LIB uSIDs have 3 MSBs = 111).

In [23]:
sets_per_block = 2 ** (set_size - block_size)
sets_per_gib = 0
for i in range(sets_per_block):
    if i >> (set_size - block_size - 3) & 7 == 7:
        # assume default GIB/LIB
        # LIB contains SIDs with 3 MSBs = 111
        sets_per_gib = i
        break
usids_per_set = 2 ** (usid_size - (set_size - block_size))

print(f'Total Sets per Block (incl. LIB): {sets_per_block}')
print(f'Sets per GIB: {sets_per_gib}')
print(f'uSIDs per set: {usids_per_set}')

Total Sets per Block (incl. LIB): 256
Sets per GIB: 224
uSIDs per set: 256


----
- Display the Set prefixes of the first Block (as example). The Set prefixes for the other Blocks are similar (typically the same Set prefix length is used). Per Set, display first and last uSID prefix.

In [24]:
block0 = next(srv6_space.subnets(new_prefix=block_size))
print(f'Set allocation example for Block [0] ({block0})')
for idx, set_pfx in enumerate(ipaddress.IPv6Network(block0).subnets(new_prefix=set_size)):
    usids = [usid.network_address 
             for usid in ipaddress.IPv6Network(set_pfx).subnets(new_prefix=block_size+usid_size)]
    print(f'Set [{idx}] {set_pfx} - uSIDs {usids[0]} to {usids[-1]}')
    if idx >= sets_per_gib-1:
        break

Set allocation example for Block [0] (fcbb:bb00::/32)
Set [0] fcbb:bb00::/40 - uSIDs fcbb:bb00:: to fcbb:bb00:ff::
Set [1] fcbb:bb00:100::/40 - uSIDs fcbb:bb00:100:: to fcbb:bb00:1ff::
Set [2] fcbb:bb00:200::/40 - uSIDs fcbb:bb00:200:: to fcbb:bb00:2ff::
Set [3] fcbb:bb00:300::/40 - uSIDs fcbb:bb00:300:: to fcbb:bb00:3ff::
Set [4] fcbb:bb00:400::/40 - uSIDs fcbb:bb00:400:: to fcbb:bb00:4ff::
Set [5] fcbb:bb00:500::/40 - uSIDs fcbb:bb00:500:: to fcbb:bb00:5ff::
Set [6] fcbb:bb00:600::/40 - uSIDs fcbb:bb00:600:: to fcbb:bb00:6ff::
Set [7] fcbb:bb00:700::/40 - uSIDs fcbb:bb00:700:: to fcbb:bb00:7ff::
Set [8] fcbb:bb00:800::/40 - uSIDs fcbb:bb00:800:: to fcbb:bb00:8ff::
Set [9] fcbb:bb00:900::/40 - uSIDs fcbb:bb00:900:: to fcbb:bb00:9ff::
Set [10] fcbb:bb00:a00::/40 - uSIDs fcbb:bb00:a00:: to fcbb:bb00:aff::
Set [11] fcbb:bb00:b00::/40 - uSIDs fcbb:bb00:b00:: to fcbb:bb00:bff::
Set [12] fcbb:bb00:c00::/40 - uSIDs fcbb:bb00:c00:: to fcbb:bb00:cff::
Set [13] fcbb:bb00:d00::/40 - uSIDs fcbb:b

Set [200] fcbb:bb00:c800::/40 - uSIDs fcbb:bb00:c800:: to fcbb:bb00:c8ff::
Set [201] fcbb:bb00:c900::/40 - uSIDs fcbb:bb00:c900:: to fcbb:bb00:c9ff::
Set [202] fcbb:bb00:ca00::/40 - uSIDs fcbb:bb00:ca00:: to fcbb:bb00:caff::
Set [203] fcbb:bb00:cb00::/40 - uSIDs fcbb:bb00:cb00:: to fcbb:bb00:cbff::
Set [204] fcbb:bb00:cc00::/40 - uSIDs fcbb:bb00:cc00:: to fcbb:bb00:ccff::
Set [205] fcbb:bb00:cd00::/40 - uSIDs fcbb:bb00:cd00:: to fcbb:bb00:cdff::
Set [206] fcbb:bb00:ce00::/40 - uSIDs fcbb:bb00:ce00:: to fcbb:bb00:ceff::
Set [207] fcbb:bb00:cf00::/40 - uSIDs fcbb:bb00:cf00:: to fcbb:bb00:cfff::
Set [208] fcbb:bb00:d000::/40 - uSIDs fcbb:bb00:d000:: to fcbb:bb00:d0ff::
Set [209] fcbb:bb00:d100::/40 - uSIDs fcbb:bb00:d100:: to fcbb:bb00:d1ff::
Set [210] fcbb:bb00:d200::/40 - uSIDs fcbb:bb00:d200:: to fcbb:bb00:d2ff::
Set [211] fcbb:bb00:d300::/40 - uSIDs fcbb:bb00:d300:: to fcbb:bb00:d3ff::
Set [212] fcbb:bb00:d400::/40 - uSIDs fcbb:bb00:d400:: to fcbb:bb00:d4ff::
Set [213] fcbb:bb00:d500:

----
- Display more information about this uSID allocation.

In [25]:
print(f'SRv6 Space prefix: {srv6_space}')
print(f'Number of /{block_size} Blocks in this space: {nr_of_blocks}')
print(f'Number of /{set_size} Sets per Block: {sets_per_gib}')
print(f'Number of uSIDs per Set: {usids_per_set}')
print(f'Number of algos: {nr_of_algos}')
print(f'Number of Blocks per algo: {nr_of_blocks // nr_of_algos}')
if nr_of_blocks % nr_of_algos:
    print(f'Number of leftover/unusable Blocks: {nr_of_blocks % nr_of_algos}')
print(f'Total number of uSIDs per Block: {usids_per_set * sets_per_gib}')
#print(f'Total number of uSIDs for all Blocks: {usids_per_set * sets_per_gib * nr_of_blocks}')
print(f'Total number of uSIDs for all usable Blocks: {usids_per_set * sets_per_gib * nr_of_algos * nr_of_blocks_per_algo}')
print(f'Total number of uSIDs for all unusable Blocks: {usids_per_set * sets_per_gib * (nr_of_blocks - (nr_of_algos * nr_of_blocks_per_algo))}')

SRv6 Space prefix: fcbb:bb00::/24
Number of /32 Blocks in this space: 63
Number of /40 Sets per Block: 224
Number of uSIDs per Set: 256
Number of algos: 2
Number of Blocks per algo: 31
Number of leftover/unusable Blocks: 1
Total number of uSIDs per Block: 57344
Total number of uSIDs for all usable Blocks: 3555328
Total number of uSIDs for all unusable Blocks: 57344


----
## Export data to excel

In [26]:
import xlsxwriter
workbook = xlsxwriter.Workbook('/mnt/c/Users/kmichiel/uSID_allocation.xlsx')
sheet_blocks = workbook.add_worksheet('Blocks')
sheet_sets = workbook.add_worksheet('Sets')

bold = workbook.add_format({'bold': True})

# export Blocks
row = col = 0
sheet_blocks.write(row, col, 'Block ID', bold)
sheet_blocks.write(row, col+1, 'Algo ID', bold)
sheet_blocks.write(row, col+2, 'Block prefix', bold)
row += 1
algo_count = [0]*nr_of_algos
for idx, block in enumerate(srv6_space.subnets(new_prefix=block_size)):
    # which algo?
    if algo_in_lsb:
        algo = idx & (nr_of_algos - 1)
    else:
        algo = (idx >> (bits_for_blocks - bits_for_algos)) & (nr_of_algos - 1)
    algo_count[algo] += 1
    # reach max number of Blocks per algo?
    if algo_count[algo] > (nr_of_blocks // nr_of_algos):
        algo = "leftover"
    sheet_blocks.write(row, col, idx)
    sheet_blocks.write(row, col+1, algo)
    sheet_blocks.write(row, col+2, str(block))
    sheet_blocks.set_column('C:C', 30)
    if idx >= nr_of_blocks-1:
        break
    row += 1

# export Sets
row = col = 0
block0 = next(srv6_space.subnets(new_prefix=block_size))
sheet_sets.write(row, col, f'Set allocation example for Block [0] ({str(block0)})')
row += 1
sheet_sets.write(row, col, 'Set ID', bold)
sheet_sets.write(row, col+1, 'Set prefix', bold)
sheet_sets.set_column('B:B', 30)
sheet_sets.write(row, col+2, 'first uSID', bold)
sheet_sets.set_column('C:C', 30)
sheet_sets.write(row, col+3, 'last uSID', bold)
sheet_sets.set_column('D:D', 30)
row += 1
for idx, set_pfx in enumerate(ipaddress.IPv6Network(block0).subnets(new_prefix=set_size)):
    usids = [usid.network_address 
             for usid in ipaddress.IPv6Network(set_pfx).subnets(new_prefix=block_size+usid_size)]
    sheet_sets.write(row, col, idx)
    sheet_sets.write(row, col+1, str(set_pfx))
    sheet_sets.write(row, col+2, str(usids[0]))
    sheet_sets.write(row, col+3, str(usids[-1]))
    if idx >= sets_per_gib-1:
        break
    row += 1

workbook.close()